In [ ]:
library(opploansanalytics)
load.packages()

In [ ]:
getLeadID = function (timestart = Sys.Date() - 1, timeend = Sys.Date(), admethod = NA, limit = NA, write.csv = FALSE, write.db = FALSE) {

    df = queryReporting(paste0(
"
select
    c_app.name as loanid
    , case  when position('::' in raw_lead ->> 'campaign_id') > 0
            then substring(raw_lead ->> 'campaign_id' from '(.*)::')
            else coalesce(
                raw_lead ->> 'campaign_id',
                raw_lead ->> 'campaignId',
                raw_lead ->> 'click_id',
                'None'
            )
            end as sub_id
from
    cloudlending.applications as c_app
    inner join
        cloudlending.advertising_method as c_adm
        on c_app.advertising_method = c_adm.id
        ", ifelse(is.na(admethod), '', paste0("and c_adm.name = '", admethod, "'")), "
    inner join
        lde4.leads as lde
        on c_app.lde4_lead_id = lde.lead_id
where
    c_app.createddate >= '", timestart, "'
    and c_app.createddate < '", timeend, "'
", ifelse(is.na(limit), '', paste0('limit ', limit)), "
"
    )) %>% 
    mutate(
        sub_id = ifelse(
            sub_id %>% str_detect('::'),
            sub_id %>% str_match_all('(.*?)::') %>%
                .[[1]] %>% .[,2] %>% as.character(),
            sub_id
        )
    )
    
    
    if (write.csv) {
        df %>% write.csv(paste0("..\\data\\subid-", timestart, ".csv"))
    }
    
    if (write.db) {
        df %>% convertDFToInsertString() %>% insertToTable('jchang.partner_subids')
    }
    
    return(df)
    
}

In [ ]:
createSubIdTable = function () {

#     dropTable('jchang.partner_subids')
    
    table.specs.df = data.frame(
        colname = c(
            'loanid',
            'sub_id'
        ),
        type = c(
            'varchar(20000)',
            'varchar(20000)'
        ),
        stringsAsFactors = FALSE
    )
    
    createTable(
        table.specs.df,
        'jchang.partner_subids'
    )
}

In [ ]:
csvs = c(
    '2019-01-01',
    '2019-02-01',
    '2019-03-01',
    '2019-04-01',
    '2019-05-01',
    '2019-06-01',
    '2019-07-01',
    '2019-08-01',
    '2019-09-01',
    '2019-10-01',
    '2019-11-01',
    '2019-12-01',
    '2020-01-01',
    '2020-02-01'
)

all = do.call(
    rbind,
    lapply(
        paste0("..\\data\\subid-", csvs, ".csv"),
        read_csv
    )
)

In [ ]:
min = 1
max = nrow(all)
i = min
counter = min

pb.rows = txtProgressBar(
    min = min,
    max = max,
    style = 3
)

while (i <= max) {
    
    all %>% 
        select(
            -X1
        ) %>% 
        slice(
            i:(i + 20000 - 1)
        ) %>% 
        convertDFToInsertString() %>% 
        insertToTable(
            'jchang.partner_subids'
        )
    
    i = i + 20000
    
    pb.rows %>% setTxtProgressBar(i)
    
}